In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import errno
%matplotlib inline
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf
plt.rcParams["figure.figsize"] = (15,10)

In [74]:
root = '/content/drive/MyDrive/Semester 5 - Project REAPS/Documents/6 sem/Code/Data/'
df = pd.read_csv(root+'ISRO_MODE/FINAL/RAIN_FALL.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'], infer_datetime_format=True)
# df = df.drop(['PREDICTED', '% ERROR'], axis = 1)
df.describe()

,ACTUAL,PREDICTED,% ERROR
count,37866.000000,37866.000000,3.786600e+04
mean,79.454423,79.514843,inf
std,99.718434,93.001666,NaN
min,0.000000,0.865485,-9.806894e+01
25%,6.333333,9.523439,-4.471202e+00
50%,18.500000,25.450701,3.337963e+00
75%,150.000000,137.802795,5.112840e+01
max,979.000000,448.207180,inf


In [75]:
df

,DATETIME,ACTUAL,PREDICTED,% ERROR
0,2015-11-06 10:30:00,150.5,144.009170,-4.312844
1,2015-11-06 11:30:00,150.5,150.011050,-0.324885
2,2015-11-06 12:30:00,150.5,160.018510,6.324591
3,2015-11-06 13:30:00,150.5,170.400310,13.222800
4,2015-11-06 14:30:00,150.5,192.560530,27.947197
...,...,...,...,...
37861,2020-04-30 23:30:00,0.0,5.731193,inf
37862,2020-05-01 00:30:00,0.5,3.435351,587.070131
37863,2020-05-01 01:30:00,0.0,3.108406,inf
37864,2020-05-01 02:30:00,0.0,2.048964,inf


In [76]:
df = df.resample('2min')

TypeError: ignored

In [65]:
gen_dataset = pd.DataFrame()

In [66]:
gen_dataset['DATETIME'] = pd.date_range("2015-11-06 10:30:00","2020-05-01 03:30:00",  freq='2min')
datelist = gen_dataset['DATETIME']

In [67]:
gen_dataset

,DATETIME
0,2015-11-06 10:30:00
1,2015-11-06 10:32:00
2,2015-11-06 10:34:00
3,2015-11-06 10:36:00
4,2015-11-06 10:38:00
...,...
1179146,2020-05-01 03:22:00
1179147,2020-05-01 03:24:00
1179148,2020-05-01 03:26:00
1179149,2020-05-01 03:28:00


In [68]:
gen_dataset = pd.merge(gen_dataset, df, on="DATETIME", how="outer")

In [69]:
gen_dataset

,DATETIME,ACTUAL
0,2015-11-06 10:30:00,150.5
1,2015-11-06 10:32:00,NaN
2,2015-11-06 10:34:00,NaN
3,2015-11-06 10:36:00,NaN
4,2015-11-06 10:38:00,NaN
...,...,...
1179146,2020-05-01 03:22:00,NaN
1179147,2020-05-01 03:24:00,NaN
1179148,2020-05-01 03:26:00,NaN
1179149,2020-05-01 03:28:00,NaN


In [73]:
gen_dataset = df.drop(['DATETIME'], axis = 1)

KeyError: ignored

In [71]:
gen_dataset.interpolate(method='linear', inplace=True)

In [72]:
gen_dataset

,ACTUAL
0,150.5
1,150.5
2,150.5
3,150.5
4,150.5
...,...
37861,0.0
37862,0.5
37863,0.0
37864,0.0
